In [1]:
#!/usr/bin/env perl
use strict;
use warnings;
use feature qw/ say current_sub /;
use Data::Dumper;
use lib '.';
use toolkit;
use graphql;
use adventofcode;

$Data::Dumper::Sortkeys = 1;

1

In [2]:
my $practice_input = qq/
89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732
/;


89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732


In [98]:
sub subp { my ($a, $b) = @_; return [ map { $a->[$_] - $b->[$_] } 0 .. $#$a ]; }
sub addp { my ($a, $b) = @_; return [ map { $a->[$_] + $b->[$_] } 0 .. $#$a ]; }
sub stringp { my ($p) = $_[0] // $_; return $p->[0].','.$p->[1] }
sub in_bounds {
    my ($p, @shape) = @_;
    return $p->[0] >= 0 && $p->[1] >= 0 && $p->[0] < $shape[0] && $p->[1] < $shape[1];
}
sub adjacent_points {
    my ($p, @shape) = @_;
    my @adj;
    return grep in_bounds($_, @shape), map addp($p, $_), [-1,0],[1,0],[0,-1],[0,1];
}
$graphql::graphql_methods{is_connected} = sub {
        my ($self, $params) = @_;
        my @row = @{$self->{arr}[$self->{coord}[0]]};
        return abs($self->{value}) ;
    };
$graphql::graphql_methods{spf_search} = sub {
        my ($self, $params) = @_;
        my $arr = $self->{arr};
        my @chains = ({ value => $self->{value}, chain => [ $self->{coord} ] });
        my @shape = shape($self->{arr});
        my $steps = 0;
        while (@chains && $steps < 9) {
            @chains =
                map { my ($chain, $next, $next_val) = @$_; { value => $next_val, chain => [ @{$chain->{chain}}, $next ] } }
                grep { my ($chain, $next, $next_val) = @$_; $next_val == ($chain->{value} + 1) }
                map { my $chain = $_; map [ $chain, $_, $arr->[$_->[0]][$_->[1]] ], adjacent_points($_->{chain}[-1], @shape) }
                @chains;
            $steps++;
        }
        return { chains => [ @chains ] };
    };

sub process_day10_part1 {
    my ($input) = @_;

    # string_2d_map_array
        # map_nd sub { $_ == 9 ? $_ : '.' },
    my $active_chains =
        map_nd selector('where.spf_search'),
        map_nd_indexed graphql_query(qq/{ where(value:0) { spf_search } }/),
        parse_2d_map_array($input);
    my @stepped_points =
        grep defined,
        flatten_nd
        map_nd sub { defined ? uniq map stringp, map @{$_->{chain}}, @{$_->{chains}} : undef },
        # map_nd sub { return unless defined; die "wat: ", Dumper $_ unless ref $_ eq 'HASH' },
        $active_chains;
    my %stepped_points_map;
    @stepped_points_map{@stepped_points} = ();
    my $debug_map =
        string_2d_map_array
        map_nd_indexed sub { my ($arr, $p) = @_; exists $stepped_points_map{stringp($p)} ? $arr->[$p->[0]][$p->[1]] : '.' },
        parse_2d_map_array($input);
    say "debug: map:\n$debug_map";
    my @path_counts =
        map scalar @$_,
        map { [ uniq map stringp, map $_->{chain}[-1], @{$_->{chains}} ] }
        grep defined,
        flatten_nd
        $active_chains;

    sum @path_counts
}
# say Dumper process_day10_part1(qq/
# 10..9..
# 2...8..
# 3...7..
# 4567654
# ...8..3
# ...9..2
# .....01
# /);
say Dumper process_day10_part1($practice_input);

debug: map:
.9010123
.8121874
87430965
96549874
45678903
32019012
01329801
.04567..
$VAR1 = 36;



1

Warning: Subroutine subp redefined at reply input line 1.

Subroutine addp redefined at reply input line 2.

Subroutine stringp redefined at reply input line 3.

Subroutine in_bounds redefined at reply input line 4.

Subroutine adjacent_points redefined at reply input line 8.

Subroutine process_day10_part1 redefined at reply input line 35.


In [99]:
my $input = get_challenge('2024/day/10/input');
my $res = process_day10_part1($input);

debug: map:
76512345.010...012965.654..65430121012345..567.7654.
83409876.3212345038743783087.521.34509256..438.85432
92056765.45694.65450321921965678.45678107.0129.91201
01143234566785078961230.01054329..498..98.87...00323
.0232109878976123876549812349.1...35430.90968901.414
432130.0107678874..7678765478.09.1267210.1.574326505
56.045.1214569965..89.9876765898.0.988761234654.7876
.78.96543234876..980187.45854767010.796501989.89.9..
98908743210.9450.671296.3090365432.768743207..743987
0781.0301.5673214589345.2101270121085401221678652156
169201292349821236701298..01989789098320034569.6.04.
254312.87.321003450..34776528766521.701.123430.76530
34322345665432343218945689439054432.658765.321089421
..910010.78.21454307654108789123211014989432..989321
018943232.9.10567...9.321767010.30..23670.01120789.0
3276593410....47890781011054321.4501.454198703163210
456.782561267832781652..212376543672..4321765425430.
968760167035890169234312309889012783.98450.2343489..
8794512385443210543..0004567..432.

652

In [100]:
say post_answer('2024/day/10/answer', 1, $res); # 652

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 10 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1

In [129]:

sub process_day10_part2 {
    my ($input) = @_;

    my $active_chains =
        map_nd selector('where.spf_search'),
        map_nd_indexed graphql_query(qq/{ where(value:0) { spf_search } }/),
        parse_2d_map_array($input);
    # my @stepped_points =
    #     grep defined,
    #     flatten_nd
    #     map_nd sub { defined ? uniq map stringp, map @{$_->{chain}}, @{$_->{chains}} : undef },
    #     $active_chains;
    # my %stepped_points_map;
    # @stepped_points_map{@stepped_points} = ();
    # my $debug_map =
    #     string_2d_map_array
    #     map_nd_indexed sub { my ($arr, $p) = @_; exists $stepped_points_map{stringp($p)} ? $arr->[$p->[0]][$p->[1]] : '.' },
    #     parse_2d_map_array($input);
    # say "debug: map:\n$debug_map";
    my @grouped_chains = 
        # map { my $chain_groups = $_; map { { chains => $chain_groups->{$_}, chain_count => scalar @{$chain_groups->{$_}} } } keys %$chain_groups }
        # map { my $chain_groups = $_; map { { chain_count => scalar @{$chain_groups->{$_}} } } keys %$chain_groups }
        # map { { group_by sub { stringp($_->{chain}[-1]) }, @{$_->{chains}} } }
        map { scalar @{$_->{chains}} }
        grep defined, flatten_nd
        $active_chains;
    # my @path_counts =
    #     map scalar @$_,
    #     map { [ uniq map stringp, map $_->{chain}[-1], @{$_->{chains}} ] }
    #     grep defined,
    #     flatten_nd
    #     $active_chains;

    sum
        # map $_->{chain_count},
        @grouped_chains
}
say Dumper process_day10_part2(qq/
..90..9
...1.98
...2..7
6543456
765.987
876....
987....
/);
say Dumper process_day10_part2(qq/
012345
123456
234567
345678
4.6789
56789.
/);
say Dumper process_day10_part2($practice_input);

$VAR1 = 13;

$VAR1 = 227;

$VAR1 = 81;



1

Warning: Subroutine process_day10_part2 redefined at reply input line 2.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Argument "." isn't numeric in numeric eq (==) at reply input line 27.

Arg

In [127]:
my $input = get_challenge('2024/day/10/input');
my $res = process_day10_part2($input);

debug: map:
76512345.010...012965.654..65430121012345..567.7654.
83409876.3212345038743783087.521.34509256..438.85432
92056765.45694.65450321921965678.45678107.0129.91201
01143234566785078961230.01054329..498..98.87...00323
.0232109878976123876549812349.1...35430.90968901.414
432130.0107678874..7678765478.09.1267210.1.574326505
56.045.1214569965..89.9876765898.0.988761234654.7876
.78.96543234876..980187.45854767010.796501989.89.9..
98908743210.9450.671296.3090365432.768743207..743987
0781.0301.5673214589345.2101270121085401221678652156
169201292349821236701298..01989789098320034569.6.04.
254312.87.321003450..34776528766521.701.123430.76530
34322345665432343218945689439054432.658765.321089421
..910010.78.21454307654108789123211014989432..989321
018943232.9.10567...9.321767010.30..23670.01120789.0
3276593410....47890781011054321.4501.454198703163210
456.782561267832781652..212376543672..4321765425430.
968760167035890169234312309889012783.98450.2343489..
8794512385443210543..0004567..432.

1432

In [128]:
say post_answer('2024/day/10/answer', 2, $res); # 1432

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 10 - Advent of Code 2024</title>
<link rel="stylesheet" type="text/css" href="/static/style.css?31"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?1" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you unlock them for real.

Please be careful with automated requests; I'm not a massive company, and I can
only take so much traffic.  Please be considerate so that everyone gets to play.

If you're

1